# Reproducing Jamie's analysis and testing  figures

Some of the code below was written by Jamie Griffith.  I am testing that here.  

I am also using this notebook as a testing ground for figures that could be included in the dashboard.

In [ ]:
library(lurn)
library(ggplot2)
library(GGally)
library(gt)
library(ggradar)
library(viridis)
library(dplyr)
library(tidyr)

In [ ]:
syn <- read.csv("../data/LURN_SI29_Synth_Final_NoFmt.csv", header = TRUE)
names(syn)[names(syn) == "Sex"] <- "SEX"
syn <- score_lurn_si_29(syn)
colnames(syn)

In [ ]:
splom_vars <- c(
   "lurn_si_29_total_score", "lurn_si_29_incontinence_score",
   "lurn_si_29_pain_score", "lurn_si_29_voiding_score",
   "lurn_si_29_urgency_score", "lurn_si_29_nocturia_score",
   "lurn_si_29_bother")

splom_dat <- syn[splom_vars]
names(splom_dat)<- c("Total","Incontinence", "Pain", "Voiding", "Urgency", "Nocturia", "Bother")
colnames(splom_dat)
head(splom_dat)

In [ ]:
ggpairs(
    splom_dat,
    lower = list(continuous = wrap("smooth_loess", alpha = 1 / 10)),
    columnLabels = c("Total", "Incontinence", "Pain", "Voiding", "Urgency",
    "Nocturia", "Bother"),
    title = "Scatterplot matrix (SPLOM) of LURN SI-29 scales: Synthetic data"
) +
theme_bw()

In [ ]:
ggplot(splom_dat, aes(x = Voiding)) + geom_density()

In [ ]:
si_29_m_nms <- lurn_si_29_names("male")
si29_prelim_freq_table <- t(apply(syn[si_29_m_nms],
                                  2,
                                  function(x) table(factor(x, levels = 0:4),
                                                    useNA = "always")))

si29_prelim_prop_table <- prop.table(si29_prelim_freq_table, margin = 1)

colnames(si29_prelim_prop_table) <- c("0", "1", "2", "3", "4", "Missing")

si29_prelim_freq_table

In [ ]:
this_patient_row <- syn[100, si_29_m_nms]
this_patient_row

In [ ]:
syn[, si_29_m_nms[1]]

In [ ]:
for (cc in si_29_m_nms){
    print(median(syn[, cc], na.rm = TRUE))
}

Long file

In [ ]:
syn_long <- read.csv("../data/syn_long.csv", header = TRUE)
patient_cells <- list()
syn_long$ID <- factor(syn_long$ID)

breaks <- seq(0, 24, 4)
labels <- c("0\nbaseline", "4\n(post-surgery)", "8", "12", "16", "20", "24")

# Spaghetti Plots
ggplot(data = syn_long,
                   aes(Week,
                       lurn_si_29_total_score,
                       group = ID,
                       colour = ID)) +
  geom_line(show.legend = FALSE) +
  geom_smooth(data = syn_long,
              aes(Week,
                  lurn_si_29_total_score),
              method = "loess",
    show.legend = FALSE,
    se = TRUE,
    inherit.aes = FALSE,
    color = "black") + 
  scale_y_continuous("LURN SI-29 Total", limits = c(0, 100)) +
  scale_x_continuous(breaks = breaks, labels = labels) +
  ggtitle("Spaghetti plot with loess smoother")

In [ ]:
head(syn_long)

In [ ]:
colnames(syn_long)
id = 3
patient_long <- syn_long[syn_long$ID == id, ]

patient_radar_data <- patient_long[c("Week",splom_vars)]
patient_radar_data$Week <- as.factor(patient_radar_data$Week)
names(patient_radar_data)<- c("Time", "Total","Incontinence", "Pain", "Voiding", "Urgency", "Nocturia", "Bother")
# I need to find a way to handle NaN values for the radar plot
patient_radar_data[is.na(patient_radar_data)] = 0
patient_radar_data 

In [ ]:
options(jupyter.plot_mimetypes = "image/png", repr.plot.width = 7, repr.plot.height = 6., repr.plot.res = 300)

ggradar(patient_radar_data, grid.min = 0, grid.mid = 50, grid.max = 100, gridline.mid.colour = "grey",
        label.gridline.mid = TRUE,
        group.colours = viridis(length(patient_radar_data) - 1),
        background.circle.transparency = 0,
        legend.title = "Weeks since procedure", 
        plot.extent.x.sf = 1.3,
        axis.label.size = 3, 
        grid.label.size = 3,
        group.point.size = 2,
        group.line.width = 0.5,
    ) + 
    theme(
        text = element_text(size = 12), 
        legend.text = element_text(size = 7),
        legend.position = c(0.75, -0.35),  
        legend.justification = c(1, 0),
        plot.margin = grid::unit(c(-0.2,0,0,0), "null")
    )  + 
    guides(color = guide_legend(nrow = 2))

In [ ]:
# read in the patient database
syn <- read.csv("../data/LURN_SI29_Synth_Final_NoFmt.csv", header = TRUE)
names(syn)[names(syn) == "Sex"] <- "SEX"
syn <- score_lurn_si_29(syn)
splom_vars <- c(
   "lurn_si_29_total_score", "lurn_si_29_incontinence_score",
   "lurn_si_29_pain_score", "lurn_si_29_voiding_score",
   "lurn_si_29_urgency_score", "lurn_si_29_nocturia_score",
   "lurn_si_29_bother")

# simplified dataframe for plots
splom_dat <- syn[c("ID", splom_vars)]
names(splom_dat)<- c("ID", "Total", "Incontinence", "Pain", "Voiding", "Urgency", "Nocturia", "Bother")

# read in the time series file
syn_long <- read.csv("../data/syn_long.csv", header = TRUE)
patient_cells <- list()
syn_long$ID <- factor(syn_long$ID)
syn_long_dat <- syn_long[c("ID", "Week", splom_vars)]
names(syn_long_dat)<- c("ID", "Week", "Total", "Incontinence", "Pain", "Voiding", "Urgency", "Nocturia", "Bother")

In [ ]:
head(syn_long_dat)

In [ ]:
line_df_input <- syn_long_dat[c("ID", "Week", "Total")]
line_df_summary <- line_df_input %>%
	group_by(Week) %>%
	summarise(
        mean = mean(Total, na.rm = TRUE), 
        sd = sd(Total, na.rm = TRUE),
    )

# loess regression
span <- 0.75
loess_mean <- loess(mean ~ Week, data = line_df_summary, span = span)
loess_sd <- loess(sd ~ Week, data = line_df_summary, span = span)

# new dataframe to plot
Week <- seq(min(line_df_summary$Week), max(line_df_summary$Week), by = 0.1)
line_df_plot <- data.frame(Week)
line_df_plot$mean <- predict(loess_mean, line_df_plot)
line_df_plot$hi <- line_df_plot$mean + predict(loess_sd, line_df_plot)
line_df_plot$lo <- line_df_plot$mean - predict(loess_sd, line_df_plot)

patient_line_data <- line_df_input[line_df_input$ID == 100, ]
patient_line_data$hi <- predict(loess_mean, patient_line_data) + predict(loess_sd, patient_line_data)

ggplot(data = line_df_plot, aes(x = Week)) + 
    geom_ribbon(aes(ymin = lo, ymax = hi), fill = "lightgray") +
    geom_line(aes(Week, mean), color = "black", size = 1.5) + 
    geom_line(aes(Week, hi), color = "black", linetype = "dashed", size = 1.5) + 
    geom_line(aes(Week, lo), color = "black", linetype = "dashed", size = 1.5)



In [ ]:
patient_line_data

In [ ]:
patient_id <- 100
input_params <- list(
    patient_id = patient_id, 
    patient_row = which(splom_dat$ID == patient_id)[1]
)
input_params

In [ ]:
head(splom_dat)

In [ ]:
col <- "Pain"
line_df_input <- syn_long_dat[c("ID", "Week", col)]
line_df_summary <- line_df_input %>%
    group_by(Week) %>%
    summarise(
        mean = mean(!!sym(col), na.rm = TRUE), 
        sd = sd(!!sym(col), na.rm = TRUE),
    )

# loess regression
span <- 0.75
loess_mean <- loess(mean ~ Week, data = line_df_summary, span = span)
loess_sd <- loess(sd ~ Week, data = line_df_summary, span = span)

# new dataframe to plot
Week <- seq(min(line_df_summary$Week), max(line_df_summary$Week), by = 0.1)
line_df_plot <- data.frame(Week)
line_df_plot$mean <- predict(loess_mean, line_df_plot)
line_df_plot$hi <- pmin(line_df_plot$mean + predict(loess_sd, line_df_plot), 100)
line_df_plot$lo <- pmax(line_df_plot$mean - predict(loess_sd, line_df_plot), 0)
line_df_plot

## bar chart of all the symptons

In [ ]:
symptoms <- c("Incontinence", "Pain", "Voiding", "Urgency", "Nocturia", "Bother", "Total")

In [ ]:
# Function to rescale a column to a specific range
rescale_to_range <- function(x, new_min, new_max) {
    scaled <- (x - min(x, na.rm = TRUE))/(max(x, na.rm = TRUE) - min(x, na.rm = TRUE))  # Scale to range [0, 1]
    scaled_rescaled <- ifelse(is.na(x), NA, scaled*(new_max - new_min) + new_min)  # Rescale to new range
    return(scaled_rescaled)
}
splom_dat$Bother <- rescale_to_range(splom_dat$Bother, 0, 100)


# create a summary for each of the columns for all patients as a comparison
df <- select(splom_dat, -ID)
summary_df <- data.frame(
    Median = apply(df, 2, median),
    Percentile_16 = apply(df, 2, function(x) quantile(x, probs = 0.16)),
    Percentile_84 = apply(df, 2, function(x) quantile(x, probs = 0.84))
)

# Add a row for the full 1sigma width
summary_df$Width <- summary_df$Percentile_84 - summary_df$Percentile_16

# add the symptoms in the correct order
summary_df$Symptom <- factor(rownames(summary_df), levels = symptoms)

# get the patient data for the bar chart
df_patient <- select(splom_dat[input_params$patient_row, ], -ID)
p <- as.data.frame(t(df_patient))
summary_df$Value <- p[, 1]


# Define a custom color palette
custom_palette <- c("#2780E3", "white", "#ff7518")

g <- ggplot(summary_df, aes(x = Symptom, y = Value, fill = (Value - Median)/Width)) +
    scale_x_discrete(limits = rev(levels(summary_df$Symptom))) +
    geom_bar(stat = "identity", fill = "white", color = "black", size = 1.5) + 
    geom_bar(stat = "identity") + 
    scale_fill_gradient2(
        low = custom_palette[1], 
        mid = custom_palette[2], 
        high = custom_palette[3],
        midpoint = 0, 
        limits = c(-1, 1),
        oob = scales::squish
    ) +    
    #geom_errorbar(data = summary_df, aes(y = NULL, ymin = Percentile_16, ymax = Percentile_84), width = 0.2, color = "black") +
    geom_errorbar(data = summary_df, aes(y = NULL, ymin = Median, ymax = Median), width = 0.75, color = "darkgray", size = 3) +
    # geom_point(data = summary_df, aes(y = Median), size = 3, color = "black") + 
    coord_flip() + 
    theme_classic() +
    guides(fill = guide_colorbar(title.position = "bottom",
        title.hjust = 0.5,
        title.vjust = 0,
        label.position = "bottom")
    ) +  
    theme(
        legend.position = "bottom",           
        legend.key.width = unit(2, "cm"),
        legend.title.align = 0.5,
        legend.margin = margin(t = 10)
    )    
g

In [ ]:
Symptom
summary_df


# For the time series, Try creating a "bubble chart" 

where each cell shows a circle for a given symptom in the given week for that patient, this size corresponds to the value, and the color based on a comparison to the median.  Could also show in gray unfilled circles, the medial values.

In [ ]:
syn_long_dat$Bother <- rescale_to_range(syn_long_dat$Bother, 0, 100)

In [ ]:
# there may be a more streamlined way to do this, but I don't know it!
median_values <- syn_long_dat %>%
    group_by(Week) %>%
    summarize(across(-c(ID), median, na.rm = TRUE))
median_values_t <- median_values %>%
    pivot_longer(cols = -c(Week),
        names_to = "Symptom",
        values_to = "Median")

Q1_values <- syn_long_dat %>%
    group_by(Week) %>%
    summarize(across(-c(ID), ~ quantile(., 0.16, na.rm = TRUE)))
Q1_values_t <- Q1_values %>%
    pivot_longer(cols = -c(Week),
        names_to = "Symptom",
        values_to = "Q1")

Q3_values <- syn_long_dat %>%
    group_by(Week) %>%
    summarize(across(-c(ID), ~ quantile(., 0.84, na.rm = TRUE)))
Q3_values_t <- Q3_values %>%
    pivot_longer(cols = -c(Week),
        names_to = "Symptom",
        values_to = "Q3")

patient_data <- select(syn_long_dat[syn_long_dat$ID == input_params$patient_id, ], -ID)
patient_data_t <- patient_data %>%
    pivot_longer(cols = -c(Week),
        names_to = "Symptom",
        values_to = "Value")

merged_df <- merge(median_values_t, Q1_values_t, by = c("Week", "Symptom")) %>%
    merge(Q3_values_t, by = c("Week", "Symptom")) %>%
    merge(patient_data_t, by = c("Week", "Symptom"))
    
# Add a row for the full 1sigma width
merged_df$Width <- merged_df$Q3 - merged_df$Q1
# don't allow a Width of zero!
merged_df$Width [merged_df$Width  == 0] <- 1.

# set the symptoms as a factor and in the correct order
merged_df$Symptom <- factor(merged_df$Symptom, levels = symptoms)

# Define a custom color palette
custom_palette <- c("#2780E3", "white", "#ff7518")

breaks <- seq(0, 24, 4)
labels <- c("0\n(baseline)", "4", "8", "12", "16", "20", "24")

ggplot(merged_df, aes(x = Week, y = Symptom, size = Value, fill = (Value - Median) / Width)) +
    geom_point(shape = 21, color = "black", stroke = 1.5) + 
    scale_fill_gradient2(
        low = custom_palette[1], 
        mid = custom_palette[2], 
        high = custom_palette[3],
        midpoint = 0, 
        limits = c(-1, 1),
        oob = scales::squish
    ) +    
    geom_point(aes(size = Median), shape = 1, color = "darkgray", stroke = 1.) +
    # geom_point(aes(size = Q1), shape = 1, color = "gray", stroke = 0.5) +
    # geom_point(aes(size = Q3), shape = 1, color = "gray", stroke = 0.5) +
    scale_size(range = c(1, 16), guide = "none") + 
    scale_x_continuous("Weeks since procedure", breaks = breaks, labels = labels) + 
    expand_limits(x = c(-1, 26)) + 
    theme_classic() + 
    guides(fill = guide_colorbar(title.position = "bottom",
        title.hjust = 0.5,
        title.vjust = 0,
        label.position = "bottom")
    ) +  
    theme(
        legend.position = "bottom",           
        legend.key.width = unit(2, "cm"),
        legend.title.align = 0.5,
        legend.margin = margin(t = 30)
    )   


In [ ]:
merged_df

In [ ]:
params <- list(
    patient_id = 100,
    patient_week = 0 
)

# read in the time series file
syn_long <- read.csv("../data/syn_long.csv", header = TRUE)
patient_cells <- list()
syn_long$ID <- factor(syn_long$ID)
syn_long_dat <- syn_long[c("ID", "Week", splom_vars)]
names(syn_long_dat)<- append(c("ID", "Week"), symptoms)

# grab the appropriate week for the current symptoms plot
syn_week <- syn_long[syn_long$Week == params$patient_week, ]
syn_week_dat <- syn_long_dat[syn_long_dat$Week == params$patient_week, ]

input_params <- list(
    patient_id = params$patient_id, 
    patient_week = params$patient_week, 
    patient_row = which(syn_week_dat$ID == params$patient_id)[1]
)

syn_week

In [ ]:
library(ggforce)

# Your data for x and y coordinates (replace with your actual data)
x <- c(1, 2, 3, 4, 5)
y <- c(10, 15, 20, 25, 30)

# Create a data frame for the points
df <- data.frame(x = x, y = y)

# Plot the points with an open circle and dashed line around it
ggplot(df, aes(x0 = x, y0 = y, r = 4)) +
  geom_circle(color = "blue", linetype = "dashed") +  # Open circle with a blue color
  theme_minimal()